# iPython Code to scrape zillow for listings when some parameters are defined by a URL

## Importing all the modules necessary for scraping and filling in google form

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

## URLs for Zillow (parameters filled) and Google Form

In [2]:
INITIAL_URL = "https://www.zillow.com/homes/for_rent/1-_beds/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3Anull%2C%22mapBounds%22%3A%7B%22west%22%3A-122.56276167822266%2C%22east%22%3A-122.30389632177734%2C%22south%22%3A37.69261345230467%2C%22north%22%3A37.857877098316834%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22mp%22%3A%7B%22max%22%3A3000%7D%2C%22price%22%3A%7B%22max%22%3A872627%7D%2C%22beds%22%3A%7B%22min%22%3A1%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D"
Google_Form_url = "https://forms.gle/r3XDBDeXPDKotqup6"

## Initializing Chrome Service and maximizing window

In [6]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(INITIAL_URL)
driver.maximize_window()

# Going through all the pages of the results and scraping address, link and price into lists

In [7]:
time.sleep(2)
# variable to store all the scraped data
addresses = []
links = []
prices = []

while True:
    
    # Mark an element for scrolling down from
    header = driver.find_element(By.CSS_SELECTOR, "h1.search-title")
    header.click()
    
    # scrolling down to load all cards using ActionChains and PAGE_DOWN
    for _ in range(100):
        action = ActionChains(driver)
        action.send_keys(Keys.PAGE_DOWN)
        action.perform()
        time.sleep(0.2)
        
    # Get HTML (updated) to parse
    webpage = driver.page_source
    
    # Parse HTML using BeautifulSoup
    soup = BeautifulSoup(webpage, "html.parser")
    all_listings = soup.find(name="ul", class_='with_constellation')
    listings = all_listings.find_all("li")
    
    # Refined above data (remove all random data, and non-property lists i.e. ads and other things)
    refined_listings = []
    for listing in listings:
        button = listing.find("button")
        try:
            if "Save this home" in button.getText():
                refined_listings.append(listing)
        except AttributeError:
            continue
            
    # Get address, link and price and add it to list variable above
    for rent_property in refined_listings:
        link = rent_property.select_one("a.property-card-link")
        links.append(link.get("href"))
        addresses.append(rent_property.find("address").getText())
        price_str = rent_property.find("span").getText()
        prices.append(price_str.split('+')[0])

    # Get to next page of listing
    next_chevron = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
    if bool(next_chevron.get_attribute('disabled')):
        break
    else:
        next_chevron.click()
        time.sleep(7)
driver.quit()

## Restarting chrome service to fill Google Form

In [22]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

## Modifying links to contain full address

In [26]:
# complete incomplete links
for i, link in enumerate(links):
    if "https" in link:
        pass
    else:
        link = "https://www.zillow.com" + link
        links[i] = link

## Go to Google Form and loop through all data and fill in the form

In [28]:
driver.get(url=Google_Form_url)
for i, address in enumerate(addresses):
    print(f"Form {i:03} filled.")
    address_input = driver.find_element(By.CSS_SELECTOR, "input[aria-labelledby='i1']")
    address_input.send_keys(address)
    rent_input = driver.find_element(By.CSS_SELECTOR, "input[aria-labelledby='i5']")
    rent_input.send_keys(prices[i])
    link_input = driver.find_element(By.CSS_SELECTOR, "input[aria-labelledby='i9']")
    link_input.send_keys(links[i])
    submit_buttons = driver.find_elements(By.CSS_SELECTOR, "div[role='button']")
    for button in submit_buttons:
        if "Submit" in button.text:
            button.click()
            break
    time.sleep(2)
    next_form = driver.find_element(By.LINK_TEXT, "Submit another response")
    next_form.click()
    time.sleep(2)

Form 001 filled.
Form 002 filled.
...
...
...
Form 734 filled.
Form 735 filled.
Form 736 filled.
Form 737 filled.
Form 738 filled.
Form 739 filled.
Form 740 filled.
Form 741 filled.
Form 742 filled.


## Quitting the driver

In [29]:
# Quit driver
driver.quit()

# Alternatively use pandas to beautify data and export to csv or excel sheet

## Importing pandas module needed for exporting to csv and pdf

In [30]:
import pandas as pd

## Creating the basic data frame containing Address, Price and URLs

In [31]:
rent_property_df = pd.DataFrame({
    "Sr_No": [i + 1 for i in range(len(addresses))],
    "Address": addresses,
    "Price_$": prices,
    "URL": links
})
rent_property_df

,Sr_No,Address,Price_$,URL
0,1,"Parkmerced | 3711 19th Ave, San Francisco, CA","$2,499",https://www.zillow.com/b/parkmerced-san-franci...
1,2,"50 Jones | 50 Jones St, San Francisco, CA","$2,410",https://www.zillow.com/b/50-jones-san-francisc...
2,3,"33 8th at Trinity Place | 33 8th St, San Franc...","$2,549",https://www.zillow.com/b/33-8th-at-trinity-pla...
3,4,"Ashton San Francisco Apartments, 301 Executive...","$2,879/mo",https://www.zillow.com/b/ashton-san-francisco-...
4,5,"Astella, 975 Bryant St #536, San Francisco, CA...","$2,892/mo",https://www.zillow.com/b/astella-san-francisco...
...,...,...,...,...
738,739,"206 Dorland St, San Francisco, CA 94114","$2,695/mo",https://www.zillow.com/homedetails/206-Dorland...
739,740,"6515 Telegraph Ave #13, Oakland, CA 94609","$2,195/mo",https://www.zillow.com/homedetails/6515-Telegr...
740,741,"2137, 21-37 W Portal Ave, San Francisco, CA 94127","$2,795/mo",https://www.zillow.com/b/21-37-west-portal-ave...
741,742,"145 Fell St APT 206, San Francisco, CA 94102","$2,295/mo",https://www.zillow.com/homedetails/145-Fell-St...


## Clean Data, i.e. remove dollar mark and \mo from price column

In [38]:
def get_price_int(string):
    if "/mo" in string:
        price_int = string[1:-3]
    else:
        price_int = string[1:]
    return int(price_int.replace(",", ""))
rent_property_df["Price_$"] = rent_property_df["Price_$"].apply(get_price_int)
rent_property_df

,Sr_No,Address,Price_$,URL
0,1,"Parkmerced | 3711 19th Ave, San Francisco, CA",2499,https://www.zillow.com/b/parkmerced-san-franci...
1,2,"50 Jones | 50 Jones St, San Francisco, CA",2410,https://www.zillow.com/b/50-jones-san-francisc...
2,3,"33 8th at Trinity Place | 33 8th St, San Franc...",2549,https://www.zillow.com/b/33-8th-at-trinity-pla...
3,4,"Ashton San Francisco Apartments, 301 Executive...",2879,https://www.zillow.com/b/ashton-san-francisco-...
4,5,"Astella, 975 Bryant St #536, San Francisco, CA...",2892,https://www.zillow.com/b/astella-san-francisco...
...,...,...,...,...
738,739,"206 Dorland St, San Francisco, CA 94114",2695,https://www.zillow.com/homedetails/206-Dorland...
739,740,"6515 Telegraph Ave #13, Oakland, CA 94609",2195,https://www.zillow.com/homedetails/6515-Telegr...
740,741,"2137, 21-37 W Portal Ave, San Francisco, CA 94127",2795,https://www.zillow.com/b/21-37-west-portal-ave...
741,742,"145 Fell St APT 206, San Francisco, CA 94102",2295,https://www.zillow.com/homedetails/145-Fell-St...


## Seeting index to Sr. No.

In [40]:
# This data frame can be easily sorted into whatever way you want for now
rent_property_df = rent_property_df.set_index("Sr_No")

,Address,Price_$,URL
Sr_No,,,
1,"Parkmerced | 3711 19th Ave, San Francisco, CA",2499,https://www.zillow.com/b/parkmerced-san-franci...
2,"50 Jones | 50 Jones St, San Francisco, CA",2410,https://www.zillow.com/b/50-jones-san-francisc...
3,"33 8th at Trinity Place | 33 8th St, San Franc...",2549,https://www.zillow.com/b/33-8th-at-trinity-pla...
4,"Ashton San Francisco Apartments, 301 Executive...",2879,https://www.zillow.com/b/ashton-san-francisco-...
5,"Astella, 975 Bryant St #536, San Francisco, CA...",2892,https://www.zillow.com/b/astella-san-francisco...
...,...,...,...
739,"206 Dorland St, San Francisco, CA 94114",2695,https://www.zillow.com/homedetails/206-Dorland...
740,"6515 Telegraph Ave #13, Oakland, CA 94609",2195,https://www.zillow.com/homedetails/6515-Telegr...
741,"2137, 21-37 W Portal Ave, San Francisco, CA 94127",2795,https://www.zillow.com/b/21-37-west-portal-ave...


## Export to excel and csv file for further manipulation/saving

In [42]:
!pip install openpyxl
filename = "PropertiesForRentInSanFrancisco"
rent_property_df.to_excel(f"{filename}.xlsx")
rent_property_df.to_csv(f"{filename}.csv")

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
